# Resampler Demo

In [ ]:
# Set data folder location
# See notebook get_nifti_data  to download data
data_path = '/devel'

In [ ]:
# Import required packages
import nibabel 
import matplotlib.pyplot as plt
import numpy as np
import os

import sirf.Reg as Reg

This demo uses the resampler in SIRF to regrid one scan into the geometry of another. A phantom was scanned with slices in the coronal orientation and again with slices in an oblique orientation, without moving the phantom. 
In this Notebook, we read and display the NIfTI images using `nibabel`, then read using SIRF and demonstrate a resampling of one into the geometry of the other.

In [ ]:
# Set the full file names for the data files
fpath  = os.path.join(data_path , 'nifti')
fn_cor = "OBJECT_phantom_T2W_TSE_Cor_14_1.nii" # Coronal volume, 30 slices
fn_obl = "OBJECT_phantom_T2W_TSE_OBL_19_1.nii"

ffn_cor = os.path.join(fpath, fn_cor)  # full file name
print("Full file name: ", ffn_cor)

ffn_obl = os.path.join(fpath, fn_obl)  # full file name
print("Full file name: ", ffn_obl)

In [ ]:
# Read in NIfTIs using nibabel and get the image data
n_cor = nibabel.load(ffn_cor)
n_obl = nibabel.load(ffn_obl)

nad_cor = n_cor.get_fdata()  # NIfTI image array data
nad_obl = n_obl.get_fdata()  # NIfTI image array data

In [ ]:
# Display a slice from each dataset
slc = 15 ;

slc_obl = nad_obl[:,:,slc]
plt.subplot(1,2,1, title='Oblique')
plt.imshow(slc_obl)

slc_cor = nad_cor[:,:,slc]
plt.subplot(1,2,2, title='Coronal')
plt.imshow(slc_cor)


Note distortion is visible in the Oblique scan.

In [ ]:
# Now read in using SIRF
R_obl = Reg.ImageData(ffn_obl)
R_cor = Reg.ImageData(ffn_cor)

In [ ]:
# Construct an identity transformation object
# We need an identity because we are using the resampler only to resample
# and not to apply any additional transformation. 

tm_identity = np.array([[1,0,0,0],
                        [0,1,0,0],
                        [0,0,1,0],
                        [0,0,0,1]])

TM = Reg.AffineTransformation(tm_identity)

In [ ]:
# Adapted from Richard Brown's notebook: sirf_registration

# Create a resampler and set the reference image to the Coronal object
resampler = Reg.NiftyResample()
resampler.set_reference_image(R_cor)

# Set the floating image to be resampled to the Oblique
resampler.set_floating_image(R_obl)

# Add the desired transformation (here identity)
resampler.add_transformation(TM)
resampler.set_padding_value(0)
# Use linear interpolation
resampler.set_interpolation_type_to_linear()
# Go!
resampler.process()

plt.figure()
plt.subplot(1,2,1, title='Oblique resampled to Coronal')
plt.imshow(resampler.get_output().as_array()[:,:,slc])
plt.subplot(1,2,2, title='Original Coronal')
plt.imshow(slc_cor)


The images are not identical - so is this correct? The regions where there are missing sections are OK because that part of the data was never in the oblique scan. The rectangular region in the circular phantom is a little different. It is hard to tell if this is a small error or because the distortions were different in the scans. (Distortions can be seen in the previous images).
The stripes in the resampled image could be an interpolation issue.
